In [89]:
from threeML import *
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [90]:
def powerlaw(piv=100):
    spec = Powerlaw()
    ps = PointSource('s',l=0,b=0,spectral_shape=spec)
    ps_model = Model(ps)
    ps_model.s.spectrum.main.Powerlaw.piv = piv
    
    ps_model.s.spectrum.main.Powerlaw.K.prior = Log_uniform_prior(lower_bound=1e-12, upper_bound=1e0)
    ps_model.s.spectrum.main.Powerlaw.index.prior = Uniform_prior(lower_bound=-5, upper_bound=0)
    
    return ps_model

def broken_powerlaw(piv=100):
    spec = SmoothlyBrokenPowerLaw()
    ps = PointSource('s',l=0,b=0,spectral_shape=spec)
    ps_model = Model(ps)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.pivot = piv
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.alpha.min_value = -3
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.K.max_value = 1
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.max_value=1000
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.min_value=1
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_scale.free = True
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.K.prior = Log_uniform_prior(lower_bound=5e-4, upper_bound=1e-3)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.alpha.prior = Uniform_prior(lower_bound=-2.2, upper_bound=-1.8)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.beta.prior = Uniform_prior(lower_bound=-5.0, upper_bound=-1.9)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.prior = Log_uniform_prior(lower_bound=10, upper_bound=500)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_scale.prior = Uniform_prior(lower_bound=0.0, upper_bound=1.5)
    
    
    ps_model.display(complete=True)
    return ps_model
    

In [91]:
def data_crab(e_range="30-400"):
    d = OGIPLike("crab",
                observation='spectra_Crab_Nebula.fits',
                response='spectral_response.rmf.fits')
    
    d.set_active_measurements(e_range)
    ps_data = DataList(d)
    return ps_data
    
def data_pulsar(e_range="30-400"):
    d = OGIPLike("crab",
                observation='spectra_A0535+26a.fits',
                response='spectral_response.rmf.fits')
    
    d.set_active_measurements(e_range)
    ps_data = DataList(d)
    return ps_data

In [92]:
def fit_data_model(data, model):
    ps_jl = JointLikelihood(model, data)
    best_fit_parameters_ps, likelihood_values_ps = ps_jl.fit()
    ps_jl.restore_best_fit()
    
    val = np.array(best_fit_parameters_ps["value"])
    err = np.array(best_fit_parameters_ps["error"])
    cor = ps_jl.correlation_matrix
    cov = cor * err[:, np.newaxis] * err[np.newaxis, :]
    
    return val, cov, ps_jl

def bayes_analysis(data, model):
    bayes_analysis = BayesianAnalysis(model, data)
    bayes_analysis.set_sampler("multinest")
    bayes_analysis.sampler.setup(n_live_points=800, resume=False, auto_clean=True)
    bayes_analysis.sample()
    
    return bayes_analysis.results._values, bayes_analysis.results.estimate_covariance_matrix()

In [93]:
def save_results(val, cov, name):
    with open(f"{name}.pickle", "wb") as f:
        pickle.dump((val, cov),f)

In [94]:
def low_energy_pl():
    piv = 100
    e_range="30-81.5"
    model = powerlaw(piv)
    data = data_crab(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "crab_low_energy_pl_fit")
    
def brk_pl():
    piv = 100
    e_range="30-400"
    model = broken_powerlaw(piv)
    data = data_crab(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "crab_brk_pl_fit")
    
def pulsar_pl():
    piv = 100
    e_range="30-400"
    model = powerlaw(piv)
    data = data_pulsar(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "pulsar_pl_fit")

In [95]:
low_energy_pl()

03:30:52 WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=94596;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=250233;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=436281;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=521809;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=664669;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=933385;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=363818;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=617985;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: gaussian                                                      ]8;id=83868;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=4697;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#470\470]8;;\

...

Values of -log(posterior) at the minimum:

,-log(posterior)
crab,-23.717964
total,-23.717964


Values of statistical measures:

,statistical measures
AIC,52.635928
BIC,52.565827
DIC,51.294364
PDIC,1.929672
log(Z),-19.950575


         INFO      deleting the chain directory chains                                     ]8;id=560811;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=955796;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\

In [96]:
brk_pl()

Model summary:
==============

                  N
Point sources     1
Extended sources  0
Particle sources  0

Free parameters (5):
--------------------

                                          value min_value max_value  \
s.spectrum.main.SmoothlyBrokenPowerLaw.K    1.0       0.0       1.0   
s...alpha                                  -1.0      -3.0       2.0   
s...break_energy                          300.0       1.0    1000.0   
s...break_scale                             0.5       0.0      10.0   
s...beta                                   -2.0      -5.0      -1.6   

                                                    unit  
s.spectrum.main.SmoothlyBrokenPowerLaw.K  keV-1 s-1 cm-2  
s...alpha                                                 
s...break_energy                                     keV  
s...break_scale                                           
s...beta                                                  

Fixed parameters (3):
---------------------

              value min_value max_value unit
s.position.l    0.0       0.0     360.0  deg
s.position.b    0.0     -90.0      90.0  deg
s...pivot     100.0      None      None  keV

Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

         WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=728493;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=293390;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=718666;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=227355;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=763930;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=759420;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=557630;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=707057;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

...

Values of -log(posterior) at the minimum:

,-log(posterior)
crab,-21.291879
total,-21.291879


Values of statistical measures:

,statistical measures
AIC,54.805981
BIC,60.066296
DIC,36.401451
PDIC,-11.054639
log(Z),-17.977181


         INFO      deleting the chain directory chains                                     ]8;id=722130;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=619804;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\

In [97]:
pulsar_pl()

         WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=8189;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=72848;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=215932;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=24921;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=457816;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=322336;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=942236;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=217981;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: gaussian                                                      ]8;id=754079;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=188881;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#470\470]8;;\

...

Values of -log(posterior) at the minimum:

,-log(posterior)
crab,-377.48967
total,-377.48967


Values of statistical measures:

,statistical measures
AIC,759.379341
BIC,761.972356
DIC,758.213923
PDIC,-6.598263
log(Z),-176.415433


         INFO      deleting the chain directory chains                                     ]8;id=357156;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=670820;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\